In [ ]:
from utils.utils import load_model
from prompts.generic_prompt import load_prefix, load_prefix_by_category, generate_response_interactive
from prompts.image_chat import convert_sample_to_shot_IC_prefix_interact, convert_sample_to_shot_IC_interact
import pprint
import random
pp = pprint.PrettyPrinter(indent=4)
args = type('', (), {})()
args.multigpu = False
device = 0

## To use GPT-Jumbo (178B) set this to true and input your api-key
## Visit https://studio.ai21.com/account for more info
## AI21 provides 10K tokens per day, so you can try only for few turns
api = False
api_key = ''

In [ ]:
## This is the config dictionary used to select the template converter
mapper = {
          "IC": {"shot_converter":convert_sample_to_shot_IC_prefix_interact, 
                 "shot_converter_inference": convert_sample_to_shot_IC_interact,
                 "file_data":"data/image_chat/","with_knowledge":False,
                  "shots":{1024:[0,1,5],2048:[0,1,10]},"max_shot":{1024:5,2048:10},
                  "shot_separator":"\n\n",
                  "meta_type":"all_turns_category","gen_len":50,"max_number_turns":2},
         }

In [ ]:
if api:
    from transformers import AutoTokenizer 
    tokenizer = tokenizer = AutoTokenizer.from_pretrained("gpt2")
    model = None
    max_seq = 2048
else:
    ## Load LM and tokenizer
    ## You can try different LMs: 
    ##   gpt2, gpt2-medium, gpt2-large, gpt2-xl,
    ##   EleutherAI/gpt-neo-1.3B, EleutherAI/gpt-neo-2.7B,
    ##   EleutherAI/gpt-j-6B
    ## the larger the better
    model, tokenizer, max_seq = load_model(args,"EleutherAI/gpt-neo-1.3B",device)

In [ ]:
## sample time is used to sample different prompts
## we select the zero element of the list
## to change the behaviour you could try different prompts
prefix_dict = load_prefix_by_category(tokenizer=tokenizer, 
                                      shots_value=mapper["IC"]["shots"][max_seq], 
                                      shot_converter=mapper["IC"]["shot_converter"], 
                                      file_shot=mapper["IC"]["file_data"]+"valid.json", 
                                      name_dataset="IC", with_knowledge=mapper["IC"]["with_knowledge"], 
                                      shot_separator=mapper["IC"]["shot_separator"],sample_times=2)[0]

In [ ]:
max_number_turns = mapper["IC"]["max_number_turns"]
prompt_sytle = {}
for sty in prefix_dict.keys():
    sty_name = sty.replace(" ","-").replace("(","").replace(")","").replace(",","").split("_")[0]
    prompt_sytle[sty_name] = prefix_dict[sty]

styles = ", ".join(list(prompt_sytle.keys()))
print(f"The possible styles are \n {styles}")
dialogue = {"dialogue":[],"personalities":""}
while True: 
    user_utt = input(">>> ")
    dialogue["dialogue"].append([user_utt,""])
    print("Choose a style from the list!")
    style = input(">>> ")
    if style not in prompt_sytle.keys():
        print("You have to choose a style from the list!")
        print("This time a random style is selected!")
        style = random.sample(list(prompt_sytle.keys()), 1)[0]
        print(f"You got the {style} style!")
    dialogue["personalities"] = style
    prefix_shots = prompt_sytle[style]

    prefix = prefix_shots.get(mapper["IC"]["max_shot"][max_seq])
    response = generate_response_interactive(model, tokenizer, shot_converter=mapper["IC"]["shot_converter_inference"], 
                                                dialogue=dialogue, prefix=prefix, 
                                                device=device, max_number_turns=mapper["IC"]["max_number_turns"], 
                                                with_knowledge=mapper["IC"]["with_knowledge"], 
                                                meta_type=mapper["IC"]["meta_type"], gen_len=50, 
                                                beam=1, max_seq=max_seq, eos_token_id=198, 
                                                do_sample=True, multigpu=False, api=api, api_key=api_key)
                    

    print(f"FSB ({style}) >>> {response}")
    dialogue["dialogue"][-1][1] = response
    dialogue["dialogue"] = dialogue["dialogue"][-max_number_turns:]

In [ ]:
## USE THIS ONLY WITH LOCAL MODELS ==> ELSE THE API QUOTA RUNS OUT IMMIDIATELY

max_number_turns = mapper["IC"]["max_number_turns"]
prompt_sytle = {}
for sty in prefix_dict.keys():
    sty_name = sty.replace(" ","-").replace("(","").replace(")","").replace(",","").split("_")[0]
    prompt_sytle[sty_name] = prefix_dict[sty]

styles = ", ".join(list(prompt_sytle.keys()))
dialogue = {"dialogue":[],"personalities":""}
while True: 
    user_utt = input(">>> ")
    dialogue["dialogue"].append([user_utt,""])
    items = list(prompt_sytle.keys())  # List of tuples of (key,values)
    random.shuffle(items) # shuffle the styles a bit
    for id_r, style in enumerate(items):
        dialogue["personalities"] = style
        prefix_shots = prompt_sytle[style]

        prefix = prefix_shots.get(mapper["IC"]["max_shot"][max_seq])
        response = generate_response_interactive(model, tokenizer, shot_converter=mapper["IC"]["shot_converter_inference"], 
                                                    dialogue=dialogue, prefix=prefix, 
                                                    device=device, max_number_turns=max_number_turns, 
                                                    with_knowledge=mapper["IC"]["with_knowledge"], 
                                                    meta_type=mapper["IC"]["meta_type"], gen_len=50, 
                                                    beam=1, max_seq=max_seq, eos_token_id=198, 
                                                    do_sample=True, multigpu=False, api=api, api_key=api_key)
        print(f"FSB ({style}) >>> {response}")
        if id_r == 10: break
    dialogue["dialogue"][-1][1] = response
    dialogue["dialogue"] = dialogue["dialogue"][-max_number_turns:]